# An analytical workflow for the estimation of the "transfer function" of spiking neuron model

_Prerequisites_: Basics about the theory of point processes. Membrane Equation for the single compartment model (e.g reproduce Kuhn et al. 2004).

adapted from the strategy suggested in the experimental study of Zerlaut et al., ~2016. 

Here, instead of the "fluctuating current + static conductance" presented in the paper, we apply this strategy on an input made of and excitatory and an inhibitory shotnoise with conductance-based synapses

See also the same calculus for a model of a dendritic structure and presynaptic correlations in Zerlaut & Destexhe ~2016 (a lot heavier though)


In [20]:
from sympy import *

# synaptic parameters: Weight, Time constant,  Number of synapses amd Reversal potential
Qe, Te, Ke, Ee = symbols("Qe, Te, Ke, Ee") # excitatory params
Qi, Ti, Ki, Ei = symbols("Qi, Ti, Ki, Ei") # inhibitory params

# membrane parameters, only passive
gL, Cm, El = symbols("gL, Cm, El") # inhibitory params

## quantities necessaries for the calculus
Ge_tot, Gi_tot, muG, Tm = symbols("Ge_tot, Gi_tot, muG, Tm") 
fe, fi = symbols("fe, fi") # synaptic frequencies, against which we want:
muV, sV, Tv = symbols("muV, sV, Tv") # the properties of the membrane potential properties


## 1) Properties of the membrane potential fluctuations: $\mu_V, \sigma_V, \tau_V$

In [35]:
# getting the mean membrane potential, see Kuhn et al.
Ge_tot, Gi_tot = Ke*Te*Qe*fe, Ki*Ti*Qi*fi
muG = Ge_tot+Gi_tot+gL # total conductance
Tm = Cm/muG # effective membrane time constant
Tm0 = Cm/gL # membrane time constant at rest

# then 
muV = (El*gL+Ge_tot*Ee+Gi_tot*Ei)/muG

# now variance and autocorrelation time

In [22]:
t, PSPe, PSPi = symbols("t, PSPe, PSPi") 

# we need the PSP, solve the membrane equation around muV for an input as an exponential synapse at time t=0,
# and fixing the drving force at the muV level, you get:
PSPe = Heaviside(t)*Qe/muG*Te/(Tm-Te)*(exp(-t/Tm)-exp(-t/Te))*(muV-Ee)
PSPi = Heaviside(t)*Qi/muG*Ti/(Tm-Ti)*(exp(-t/Tm)-exp(-t/Ti))*(muV-Ei)

# N.B. there seems to be an exception for Tm=Te or Tm=Ti, indeed, needs a different solving method (alpha-like solution)
# but the results become the same after Fourier transform and integration...

In [24]:
f, tfPSPe, tfPSPi = symbols("f, tfPSPe, tfPSPi") # for Fourier analysis

# doesn't work in sympy yet... :(
# but doing this whould do the job:

# tfPSPe = fourier_transform(PSPe, t, f)
# tfPSPi = fourier_transform(PSPi, t, f)

In [26]:
# so we do it manually
PSP, T, tfPSP = symbols("PSP, T, tfPSP")
PSP = Heaviside(t)*exp(-t/T)
tfPSP = fourier_transform(PSP, t, f) # this works

# then exploiting linearity and replacing
tfPSPe = Qe/muG*Te/(Tm-Te)*(muV-Ee)*(tfPSP.subs(T, Tm)-tfPSP.subs(T, Te))
tfPSPi = Qi/muG*Ti/(Tm-Ti)*(muV-Ei)*(tfPSP.subs(T, Tm)-tfPSP.subs(T, Ti))

In [ ]:
psd = symbols("psd") # power spectrum of the membrane potential fluctuations
psd = fe*Ke*Abs(tfPSPe)**2+fi*Ki*Abs(tfPSPi)**2 # psd for this point process (shotnoise theory)

In [33]:
# then variance
sV = sqrt(integrate(psd, (f, -oo, oo)))

In [ ]:
# then autocorrelation time
Tv = integrate(psd, (f, -oo, oo))/psd.subs(f, 0)/2


## 2) Spiking probability from the fluctuations properties

In [38]:
# each neuronal model is described by an effective threshold that is a 

# second order polynomial of 3 variables (muV, sV, Tv) + linear dependency on total conductance
P0, P1, P2, P3, P4, P5, P6, P7, P8, P9, P10 = symbols("P0, P1, P2, P3, P4, P5, P6, P7, P8, P9, P10") 
# scaling factors (useless, will be global variables, it's just to insure that the P coefficents are all in the same range)
muV0, DmuV0, sV0, DsV0, TvN0, DTvN0 = symbols('muV0, dmuV, sV0, dsV, Tv0, dTv')
TvN = symbols('TvN')
TvN = Tv/Tm0 # normalized autocorrelation time (because neurons can have varying time constants)

Vthre_eff = symbols('Vthre_eff') 
# second order polynomial
Vthre_eff = P0+P1*(muV-muV0)/DmuV0+\
            P2*(sV-sV0)/DsV0+P3*(TvN-TvN0)/DTvN0+\
            P4*log(muG/gL)+P5*((muV-muV0)/DmuV0)**2+\
            P6*((sV-sV0)/DsV0)**2+P7*((TvN-TvN0)/DTvN0)**2+\
            P8*(muV-muV0)/DmuV0*(sV-sV0)/DsV0+\
            P9*(muV-muV0)/DmuV0*(TvN-TvN0)/DTvN0+\
            P10*(sV-sV0)/DsV0*(TvN-TvN0)/DTvN0

# Finally, spiking probability
Fout = symbols('Fout') 
Fout = 1/2/Tv*erfc((Vthre_eff-muV)/np.sqrt(2)/sV)

In [ ]:
# then we will also need the derivatives with respect to the presynaptic activities
diff(Fout, fe)

In [ ]:
diff(Fout, fi)

In [ ]:
diff(diff(Fout, fe), fe)

In [ ]:
diff(diff(Fout, fe), fi)

In [ ]:
diff(diff(Fout, fi), fi)